# Understanding the Monty Hall Problem Using Step-by-Step Simulations

The Monty Hall problem is a classic probability puzzle named for the host of the game show that made it famous.

In the game contestants are shown three doors. Behind one door is a prize, behind the other two are nothing*.

The contestant selects a door. After the initial selection, the host opens one of the remaining two doors, invariably revealing one of the losses, thus building suspense and introducing the next phase of the puzzle.

The host then asks the question that makes this puzzle so intersting: "Would you like to stay with your guess, or switch to a new door?". At this point there is one open door (the one the host opened) and two closed (the initial guess and the remaining door). The question - does it benefit the contestant to switch to a new door?

The popular and intuitive answer, famously, is 'no' - there is a 1/3 chance of getting it right - one win, three doors. The host didn't move anything around. Why would the odds have changed?

Just as (in)famously, the *correct* answer is 'yes' - you are twice as likely to win if you switch.

How can this be?

This answer is so counterintuitive that the overwhelming number of people get it wrong initially. Something like 90% of a sample of the general public got the question wrong. Even among academics, many of them mathematicians, barely over 1/3 got the answer right (** *).


In [1]:
import random

The first thing we need is the potential outcomes of the game - two losses and a win. It's easy to represent these in a list and shuffle it to generate a random game.

In [2]:
# potential outcomes
outcomes = ["win", "loss", "loss"]

# shuffle the results
random.shuffle(outcomes)

print(outcomes)

['win', 'loss', 'loss']


The next thing we will need is way to represent one of the base components of they game - a door. No need for anything too fancy - we'll just make it into a class to keep it organized.

In [3]:
class Door(object):

    # just something to refer to it by
    name = None

    # will have contents (win or loss)
    contents = None

    # will or will not be opened at any given time
    isOpen = False
    
    def open_(self):
        self.isOpen = True

# a simple function to open a door
def openDoor(door):
    door.isOpen = True
    return door

We can now make some doors and assign the outcomes we've generated with our shuffle. 

In [4]:
# make our doors
door1 = Door()
door2 = Door()
door3 = Door()

# give them our shuffled outcomes and name them
door1.contents = outcomes[0]
door1.name = "Door #1"

door2.contents = outcomes[1]
door2.name = "Door #2"

door3.contents = outcomes[2]
door3.name = "Door #3"

# put them all in a list, simply for convinience
doors = [door1, door2, door3]

Now that we've set up the game, we can actually play it! Which door to pick? We will just write a function to select a random.

In [5]:
def pickRandomDoor(doorList):
    return doorList[random.randint(0,2)]

# let's see if it's working
print(pickRandomDoor(doors).name)
print(pickRandomDoor(doors).name)
print(pickRandomDoor(doors).name)

Door #3
Door #2
Door #3


Looks good!

In [6]:
chosenDoor = pickRandomDoor(doors)

chosenDoor.name

'Door #3'

The die is cast. Now, the host will reveal a door, and to prevent the game from ending early, they'll make sure to *reveal a loss*. This is the key to the whole problem. We can emulate this bahavior with a function as well. 

In [7]:
def revealLoss(doors, chosenDoor):
    
    # we will look for the door to reveal in a random order...
    orderToLook = [0,1,2]

    # shuffle them
    random.shuffle(orderToLook)

    # ...but then look through each door to see if it's closed and a loss
    for number in orderToLook:
        doorAtNumber = doors[number]
        # can't choose the win or doors that are already opened!
        if doorAtNumber.isOpen is False \
            and doorAtNumber.contents is not "win" \
                and doorAtNumber is not chosenDoor:
            return doorAtNumber

Let's test this function by calling it 1000 times and making sure it always brings back losses from unopened doors. (note: a 'set' is just each unique item in a list once)

In [8]:
# call the function 1000 times
testLosses = [revealLoss(doors, chosenDoor = chosenDoor) for i in range(1000)]

# are all the open statuses False?
print(set([door.isOpen for door in testLosses]))

# are all the contents a loss
print(set([door.contents for door in testLosses]))

{False}
{'loss'}


The last thing we need is a function that allows us to switch doors if we so choose. It will need information on what doors we have (the doors list) and which door we chose initially (we can't switch to the door we're on!). It will open the door that a) isn't already opened and b) isn't the door we chose first. We can implement it thusly:

In [9]:
def switchDoors(doors, currentDoor):
    # pick a door that ...
    for d in doors:
        # ...isn't the current door...
        if d == currentDoor:
            continue # to the next "d"
        # ... and isn't opened
        if d.isOpen == True:
            continue # to the next "d"
        return d

Let's test this function in context with the others.

In [10]:
# show our first choice
print("Our first choice is", chosenDoor.name)

# switch our chosen door
chosenDoor = switchDoors(doors, chosenDoor)

print("After switching we have", chosenDoor.name)

Our first choice is Door #3
After switching we have Door #1


We can also test it in the context of a door being revealed to make sure we don't switch to a door that's been opened: 

In [11]:
# show our first choice
print("Our first choice is", chosenDoor.name)

# reveal a loss
doorOpened = openDoor(revealLoss(doors, chosenDoor))

# show which door was opened
print("The loss that we revealed was", doorOpened.name)

# switch our chosen door
chosenDoor = switchDoors(doors, chosenDoor)

print("So our choice after switching must be", chosenDoor.name)

Our first choice is Door #1
The loss that we revealed was Door #3
So our choice after switching must be Door #2


We've now get everything we need to run a simulation of the game! Let's run one. We will do it step by step to make it as clear as possible.

In [12]:
### set up the game

# will we switch?
switch = True

# potential outcomes
outcomes = ["win", "loss", "loss"]

# shuffle the results
random.shuffle(outcomes)

# set up the doors
door1 = Door()
door2 = Door()
door3 = Door()

# assign names and contents them
door1.contents = outcomes[0]
door1.name = "Door #1"

door2.contents = outcomes[1]
door2.name = "Door #2"

door3.contents = outcomes[2]
door3.name = "Door #3"

# simply for convinience
doors = [door1, door2, door3]

### make our initial selection
chosenDoor = pickRandomDoor(doors)

### reveal a loss 
doorOpened = openDoor(revealLoss(doors, chosenDoor))

### the switch!
if switch == True:
    chosenDoor = switchDoors(doors, chosenDoor)
else: # stay with our first guess
    pass

print(chosenDoor.contents)

win


To make it easy to generate lots of results, we can wrap this in a function:

In [13]:
def runMonteHall(switch = True):

    ### set up the game
    # potential outcomes
    outcomes = ["win", "loss", "loss"]

    # shuffle the results
    random.shuffle(outcomes)

    # set up the doors
    door1 = Door()
    door2 = Door()
    door3 = Door()

    # assign names and contents them
    door1.contents = outcomes[0]
    door1.name = "Door #1"

    door2.contents = outcomes[1]
    door2.name = "Door #2"

    door3.contents = outcomes[2]
    door3.name = "Door #3"

    # simply for convinience
    doors = [door1, door2, door3]

    ### make our initial selection
    chosenDoor = pickRandomDoor(doors)

    ### reveal a door 
    doorOpened = openDoor(revealLoss(doors, chosenDoor))

    ### the switch!
    if switch == True:
        chosenDoor = switchDoors(doors, chosenDoor)
    else: # stay with our first guess
        pass

    return chosenDoor

Now we can confirm our theoretical understanding of the problem by simply running thousands of games to see that pattern that emerges. To make this easy, let's write a simple function that scores lists of games for us.

In [14]:
def scoreGames(gamesList):

    # count wins vs losses
    wins = [result for result in gamesList if result == "win"]
    losses = [result for result in gamesList if result == "loss"]

    # get percents
    winPercent = round(len(wins)/len(gamesList), 4) * 100
    lossPercent = round(len(losses)/len(gamesList), 4) * 100

    # return a dict of results
    return {"win %:": winPercent, "loss %:": lossPercent}

Now we'll run a bunch of games and count how many times we won with switching versus staying:

In [15]:
# games where we switch
switchGames = [runMonteHall(switch = True).contents for i in range(5000)]

# games where we stay
stayGames = [runMonteHall(switch = False).contents for i in range(5000)]

# let's take a look
switchResults = scoreGames(switchGames)
print("With switching:", switchResults)

stayResults = scoreGames(stayGames)
print("With staying:", stayResults)

With switching: {'win %:': 66.44, 'loss %:': 33.56}
With staying: {'win %:': 33.879999999999995, 'loss %:': 66.12}



     [?] [?] [?]


|Door 1|Door 2|Door 3|
|:------|:-----:|------:|
| `1/3` | `1/3` | `1/3` |
| `[?]` | `[?]` | `[?]` |
|`picked`|`closed`|`closed`|

### Testing the revealLoss function

Just to make sure we never reveal a win or a the door we've already we opened we can do a little quality control. We'll make reveal 1,000 doors and tell Python to kick an error if it finds something we don't want. 


Specifically, we'll make sure there is only 2 (or fewer) doors ever chosen by the revealLoss function. If we choose the correct door on our first guess, the host has 2 doors to choose from (they can't open the one we chose, but neither of the other doors are the winner so they can open either). If we're wrong, the host only has one choice. They can't open our door, or the winner, leaving just one option (this will be an important insight later on, so keep it in mind). Thus, if our simulated losses contain 1 or 2 names, that's fine - but if there are three different doors in there we've done something wrong. 

In [16]:
# a test set of doors 
test1 = Door()
test1.name = "Test #1"
test1.contents = "win"

test2 = Door()
test2.name = "Test #2"
test2.contents = "loss"

test3 = Door()
test3.name = "Test #3"
test3.contents = "loss"

# make our choice - we set the winner manually for transparency
testChoice = test1

# make a list of our test doors
testDoors = [test1, test2, test3]

# generate one thousand losses
oneThousandLosses = [revealLoss(testDoors, testChoice) for i in range(1000)]

# get all the names of the doors in the test losses
lossesNames = [door.name for door in oneThousandLosses]

# the unique names found (set simply returns one copy of each unique in a list)
print("Names in the losses:", set(lossesNames))

# this will cause an error if there are more than two names in the list
assert len(set(lossesNames)) <= 2

# compare that to the chosen door
print("When", testChoice.name, "contains:", testChoice.contents)

Names in the losses: {'Test #2', 'Test #3'}
When Test #1 contains: win


We see that our function returns the two doors that aren't the winner in our test of 1000 cases